In [ ]:
from qTools import *
import Cahit as ck

import numpy as np

import matplotlib.pyplot as plt
import datetime

### System parameters

In [ ]:
resonatorDimension = 200
g = 1.79
qfreq = 0
resFreq = 1

### Simulation Parameters and Settings

In [ ]:
qSim = Simulation()
qSim.finalTime = 2
qSim.stepSize = 0.02
qSim.delStates = True

### Composite System Creation and Components

In [ ]:
JCSys = QuantumSystem(name='JC')
RabiSys = QuantumSystem(name='Rabi')

cavJC = JCSys.addSubSys(Cavity(dimension=resonatorDimension, frequency=resFreq))
qubJC = JCSys.createSubSys(Qubit, frequency=qfreq)

#cavRabi = RabiSys.addSubSys(cavJC.copy())
cavRabi = RabiSys.addSubSys(cavJC.copy(),frequency=2*resFreq)
qubRabi = RabiSys.addSubSys(qubJC.copy())

#### Coupling term and initial state

In [ ]:
JCcoupling = JCSys.JC(cavJC, qubJC, g)
RabiCoupling = RabiSys.Rabi(cavRabi, qubRabi, g)

#JCSys.initialState = [0,0]
cavJC.initialState = 0
qubJC.initialState = basis(2,0)

cavRabi.initialState = 0
qubRabi.initialState = basis(2,0)

### Define the Digital Algorithm

In [ ]:
JCunitary = freeEvolution(ratio=0.5, superSys=JCSys)
#qubset = JCunitary.createUpdate(system=qubJC, key='frequency', value=0)

bitFlip = xGate(implementation='instant', superSys=qubJC)
#AJC = freeEvolution(superSys=JCSys)

#AJCunitary = qProtocol(steps=[bitFlip, AJC, bitFlip])
digitalRabi = qProtocol(superSys=JCSys, steps=[JCunitary, bitFlip, JCunitary, JCunitary, bitFlip, JCunitary], name='DigitalRabi')

### Add the systems into Simulation interface

In [ ]:
qSim.addSubSys(JCSys, digitalRabi)
qSim.addSubSys(RabiSys)

### Define Parameter Sweeps

In [ ]:
'''cavJCFreqSweep = qSim.Sweep.createSweep(system=cavJC, sweepKey='frequency', sweepList=np.arange(-2, 2+0.005, 0.005))
cavRabiFreqSweep = qSim.Sweep.createSweep(system=cavRabi, sweepKey='frequency', sweepList=np.arange(-4, 4+0.01, 0.01))'''
stepSizeSweep = qSim.Sweep.createSweep(system=qSim, sweepKey='stepSize', sweepList=np.arange(0.001,0.1,0.001))

### Define the (run-time) compute

In [ ]:
cavParity = parityEXP(cavJC.freeMat)

def expectn(qSim, stateJC, stateRabi):
    
    #qSim.qRes.results['simFidelity'].append(fidelityKet(stateJC, stateRabi))
    qSim.qRes.resultsMethod('simFidelity', fidelityKet(stateJC, stateRabi))

    #qSim.getResultByName('RabiResults').results['cavParity'].append(expectationKet(cavParity, stateRabi))
    qSim.getResultByName('RabiResults').resultsMethod('cavParity', expectationKet(cavParity, stateRabi))
    
    #qSim.getResultByName('JCDigitalRabiResults').results['cavParity'].append(expectationKet(cavParity, stateJC))
    qSim.getResultByName('JCDigitalRabiResults').resultsMethod('cavParity', expectationKet(cavParity, stateJC))

qSim.compute = expectn

### Run the simulation

In [ ]:
nw = datetime.datetime.now()

results = qSim.run(p=True, coreCount='all')

en = datetime.datetime.now()
print(en-nw)

### Save Results

In [ ]:
path, fname = qSim.qRes.saveAll(fileName='trial', path='/Users/cahitkargi/Desktop')

### Plot the Results
---

Plotting is not part of the simulation library

In [ ]:
def plotRes(xList = cavRabiFreqSweep.sweepList):
    fig = plt.figure(figsize=(12,9))
    setC = [0.88, 0.25, 0.02, 0.6]

    axGrids = ck.Plotting.Functions.grid(3, 1, fig=fig)

    ck.Plotting.SimplePlots.colorPlot(xList, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.qRes.results['simFidelity'], ax=axGrids[0], gif=fig, setC=setC)
    ck.Plotting.SimplePlots.colorPlot(xList, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.getResultByName('RabiResults').results['cavParity'], ax=axGrids[1], gif=fig, setC=setC)
    ck.Plotting.SimplePlots.colorPlot(xList, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.getResultByName('JCDigitalRabiResults').results['cavParity'], ax=axGrids[2], gif=fig, setC=setC)

plotRes(cavRabiFreqSweep.sweepList)

In [ ]:
def plotRes(xList = stepSizeSweep.sweepList):
    fig = plt.figure(figsize=(12,9))
    setC = [0.88, 0.25, 0.02, 0.6]

    axGrids = ck.Plotting.Functions.grid(3, 1, fig=fig)

    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.qRes.results['simFidelity'], ax=axGrids[0], gif=fig, setC=setC, mapC='GrYl', maxC=1, minC=0)
    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.getResultByName('RabiResults').results['cavParity'], ax=axGrids[1], gif=fig, setC=setC)
    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.getResultByName('JCDigitalRabiResults').results['cavParity'], ax=axGrids[2], gif=fig, setC=setC)

plotRes()